## **P4 (Містобудування, ЖКГ, побутове, торговельне обслуговування, транспорт і зв’язок): 3/6**

In [1]:
import numpy as np
import pandas as pd
from custom_func import *

In [2]:
#невірний код розрахунку показників по генпланам, на видалення

'''
df_P04_005 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_005.xlsx", dtype={"КОАТУУ населеного пункту": str})
settlement = pd.read_excel(f"{INPUTS_PATH}/P99/settlements.xlsx")

urban = ["місто", "селище міського типу"]
rural = ["село",  "селище"]

df_P04_005["param_type"] = np.select(
    condlist=[
        df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(urban),
        df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(rural)
    ],
    choicelist=["p4_02_raw", "p4_03_raw"],
    default= np.nan
)

df_P04_005_fin = df_P04_005.loc[
    (df_P04_005["Дата прийняття рішення про затвердження Генерального плану населеного пункту"] <= "2020-06-30") &
    (df_P04_005["Дата прийняття рішення про затвердження Генерального плану населеного пункту"] >= "2020-04-01")
]

df_P04_005_fin = df_P04_005.groupby(["region", "param_type"])["Населений пункт"].size().reset_index()
df_P04_005_fin = df_P04_005_fin.pivot(index='region', columns='param_type').fillna(0).reset_index()
df_P04_005_fin.columns = df_P04_005_fin.columns.droplevel()
df_P04_005_fin.columns = ["region", "p4_02_raw", "p4_03_raw"]

df_P04_005_fin = pd.merge(
    settlement.rename(columns={"Кількість населених пунктів": "total"}),
    df_P04_005_fin,
    how="outer"
)

df_P04_005_fin["p4_02_raw"] = df_P04_005_fin["p4_02_raw"] / df_P04_005_fin["total"]
df_P04_005_fin["p4_03_raw"] = df_P04_005_fin["p4_03_raw"] / df_P04_005_fin["total"]

normalize_parameter(df_P04_005_fin,'p4_02_raw','p4_02',min_bound=0,max_bound=1)
normalize_parameter(df_P04_005_fin,'p4_03_raw','p4_03',min_bound=0,max_bound=1)
'''

'\ndf_P04_005 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_005.xlsx", dtype={"КОАТУУ населеного пункту": str})\nsettlement = pd.read_excel(f"{INPUTS_PATH}/P99/settlements.xlsx")\n\nurban = ["місто", "селище міського типу"]\nrural = ["село",  "селище"]\n\ndf_P04_005["param_type"] = np.select(\n    condlist=[\n        df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(urban),\n        df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(rural)\n    ],\n    choicelist=["p4_02_raw", "p4_03_raw"],\n    default= np.nan\n)\n\ndf_P04_005_fin = df_P04_005.loc[\n    (df_P04_005["Дата прийняття рішення про затвердження Генерального плану населеного пункту"] <= "2020-06-30") &\n    (df_P04_005["Дата прийняття рішення про затвердження Генерального плану населеного пункту"] >= "2020-04-01")\n]\n\ndf_P04_005_fin = df_P04_005.groupby(["region", "param_type"])["Населений пункт"].size().reset_index()\ndf_P04_005_fin = df_P04_005_fin.pivot(index

### **p4_02**
#### наявність генеральних планів у містах та селищах міського типу
### **p4_03**
#### наявність генеральних планів у селах

In [3]:
# _dict_ids = population.to_dict()["p4_region_id"]
# _dict_ids_reversed = {v:k for k, v in _dict_ids.items()}

# # датасет ОДА
# df_P04_005 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_005.xlsx", dtype={"КОАТУУ населеного пункту": str})
# df_P04_005 = df_P04_005.loc[df_P04_005["region"].ne("26_kyiv")].copy()

# urban = ["місто", "смт"]
# rural = ["село",  "селище"]

# # додаю так, щоб мати змогу по-іншому визначити urban / rural, не змінюючи оригінальні дані
# df_P04_005["param_type"] = np.select(
#     condlist=[
#         df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(urban),
#         df_P04_005["Тип населеного пункту (село, селище, селище міського типу, місто)"].isin(rural)
#     ],
#     choicelist=["p4_02_raw", "p4_03_raw"],
#     default= np.nan
# )

# df_P04_005["plan"] = np.where(df_P04_005["Наявність затвердженого Генерального плану населеного пункту (так/ні)"].eq("так"), 1,0)
# df_P04_005_fin = df_P04_005.groupby(["region_id", "param_type"], as_index=False)["plan"].mean()
# df_P04_005_fin = df_P04_005_fin.pivot(index='region_id', columns='param_type', values='plan').reset_index()
# df_P04_005_fin["region"] = df_P04_005_fin["region_id"].map(_dict_ids_reversed)
# df_P04_005_fin = df_P04_005_fin.drop("region_id", 1)

# normalize_parameter(df_P04_005_fin,'p4_02_raw','p4_02',min_bound=0,max_bound=1)
# normalize_parameter(df_P04_005_fin,'p4_03_raw','p4_03',min_bound=0,max_bound=1)

### **p4_05**
#### протяжність автомобільних доріг загального користування місцевого значення, на яких проведено капітальний ремонт, реконструкцію, відносно загальної довжини доріг

In [4]:
df_P04_006 = pd.read_excel(f"{INPUTS_PATH}/P4/P04_006.xlsx")
roads = pd.read_csv(f"{INPUTS_PATH}/P99/all_roads.csv").rename(columns={"Область": "region"})

In [5]:
df_P04_006 = df_P04_006.loc[pd.to_datetime(df_P04_006["Дата початку ремонтних робіт"]) < "2020-07-01"]

df_P04_006_fin = pd.concat([
    roads.groupby("region")["Довжина (протяжність) дороги (км)"].sum().rename('roads_all'),
    df_P04_006.groupby('region')['Довжина (протяжність) відремонтованої ділянки (км)'].sum().rename('roads_repaired'),
    df_P04_006.groupby('region')['Загальна вартість ремонту (грн)'].sum().rename('price')
], axis=1)

df_P04_006_fin['price_km'] = df_P04_006_fin['price']/df_P04_006_fin['roads_repaired']
df_P04_006_fin['p4_05_raw'] = df_P04_006_fin['roads_repaired']/df_P04_006_fin['roads_all']
df_P04_006_fin = df_P04_006_fin.reset_index().rename(columns={"index": "region"})
normalize_parameter(df_P04_006_fin,'p4_05_raw','p4_05',min_bound=0)

Емпіричні границі: (0, 0.020276599459073542)
Коефіцієнти нормалізації: (49.31793430246567, -0.0)
Параметр p4_05 нормалізовано і додано до таблиці



### **p4_06**
#### рівень використання коштів дорожнього фонду, передбачених для будівництва, реконструкції, ремонту та утримання автомобільних доріг загального користування місцевого значення, вулиць і доріг комунальної власності у населених пунктах

In [6]:
df_KMU["p4_06_raw"] = df_KMU["Дор фонд_видатки"] / df_KMU["Дор фонд_передбачено"]
normalize_parameter(df_KMU, "p4_06_raw", "p4_06")

Емпіричні границі: (0.022008531339642, 0.6319770035765523)
Коефіцієнти нормалізації: (1.6394289959491584, -0.03608142443646487)
Параметр p4_06 нормалізовано і додано до таблиці



### **p4_07**
#### середній рівень будівельної готовності об'єктів Великого будівництва (за касовими видатками)
### **p4_08**
#### середній рівень освоєння коштів, виділених на об'єкти Великого будівництва з усіх джерел

In [7]:
'''
сюди додати розрахунки за параметрами p4_07, p4_08
'''

'\nсюди додати розрахунки за параметрами p4_07, p4_08\n'

### **Група 1:** p4_02-p4_03, генплани
### **Група 2:** p4_05-p4_06, дороги
### **Група 3:** p4_07-p4_08, велике будівництво

In [8]:
dict_weights = {
    'p4_02':1,
    'p4_03':1,
    'p4_05':1,
    'p4_06':1,
    'p4_07':1,
    'p4_08':1
}

sources = [
#     df_P04_005_fin, 
    df_P04_006_fin,
    df_KMU
]
save_data(sources,regions,dict_weights,'P4')

Дані за галузевим параметром P4 збережено
               region     p4_05     p4_06        P4
0           Волинська  0.526775  0.128628  1.092339
1           Вінницька  0.318542  0.480516  1.331762
2    Дніпропетровська  0.948457  0.627844  2.627169
3            Донецька  0.276283  0.895208  1.952484
4         Житомирська  0.377780  0.289077  1.111428
5        Закарпатська  0.422385  0.102066  0.874085
6          Запорізька  0.362038  0.317821  1.133098
7            Київська  0.434851  0.531625  1.610794
8   Івано-Франківська  1.000000  0.200172  2.000287
9      Кіровоградська  0.204456  0.299042  0.839162
10          Луганська  0.653571  0.338171  1.652903
11          Львівська  0.341205  1.000000  2.235342
12       Миколаївська  0.133177  0.161522  0.491164
13            Одеська  0.410382  0.219689  1.050118
14         Полтавська  0.113465  0.670512  1.306629
15         Рівненська  0.643112  0.380143  1.705425
16            Сумська  0.059658  0.129459  0.315195
17      Тернопільська 